<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spark data frame
**bold text**

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
#sc: SparkContext= SparkSession\
spark = SparkSession\
.builder\
.appName("Spark-SQL")\
.master("local")\
.getOrCreate()

In [31]:
moviesdf=spark.read.option("header","true").option("inferschema","true").format("csv").load("/content/movies.csv")
ratingsdf=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/ratings.csv")

In [33]:
moviesdf.printSchema()
ratingsdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)

